### Computational Guided Inquiry for Modeling Earth's Climate (Neshyba, 2024)

# Cambio 3.0

## Equations of motion of Cambio3.0
The equations of motion of Cambio3.0 are the same as Cambio2.0:
$$
F_{land->atm} =  k_{la} \ \ \ (1) 
$$

$$
F_{atm->land} = k_{al0} +  k_{al1} \times [C_{atm}] \ \ \ (2)
$$

$$
F_{ocean->atm} = k_{oa} \times (1+DC\times T_{anomaly}) [C_{ocean}] \ \ \ (3)
$$

$$
F_{atm->ocean} = k_{ao} [C_{atm}] \ \ \ (4)
$$

$$
F_{human->atm} = \epsilon(t) \ \ \ (5)
$$

The difference is that ice-albedo is taken into account, within each iteration of the Euler loop, as follows:

- We'll calculate an albedo based on the temperature anomaly (for which purpose we have a CL function).
- We'll calculate a temperature anomaly based on that albedo.


## Uploading your climate emissions scenario
As before, you'll need to upload a climate emissions scenario file to the Cambio3.0 folder. Now, you should incude an emission scenario that has long-term (post-peak) emissions, generated by ScheduledFlowsWithLTE.

## Learning goals
1. I can describe the geophysical processes behind ice-albedo feedback.
1. I can implement ice-albedo feedback in an Euler loop.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5io
import sys; sys.path.append('/home'); import MECLib as CL
from copy import copy as makeacopy

In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams['font.size'] = 18

### Loading the scheduled flow
In the cell below, load in your scheduled flows file. It'll be most convenient if you use the following naming convention: 

    epstime, eps, epsdictionary = CL.GetMyScenario('...')

(but of course supplying the name of your own scheduled flows file).

Notice that the line above assigns the variable "epstime" -- that's just to make it really clear that it's the array that comes out of the scheduled flows file.

In [ ]:
# your code here 


### Creating a dictionary for climate parameters
In the cell below, we use the CreateClimateParams function to create a dictionary of climate parameters.

In [ ]:
ClimateParams = CL.CreateClimateParams(epsdictionary)
display(ClimateParams)

### Duplicating Cambio2.0
Below, we reproduce Cambio2.0.

In [ ]:
# This is the common title
plot_title = 'Cambio2.0'

def PropagateClimateState_Cambio2(previousClimateState, ClimateParams, dt, F_ha):
    """Propagates the state of the climate, with a specified anthropogenic carbon flux"""
    """Returns a new climate state"""

    # Extract constants from ClimateParams
    k_la = ClimateParams['k_la']
    k_al0 = ClimateParams['k_al0']
    k_al1 = ClimateParams['k_al1']
    k_oa = ClimateParams['k_oa']
    k_ao = ClimateParams['k_ao']
    DC = ClimateParams['DC']
    
    # Extract concentrations, albedo, etc, from the previous climate state
    C_atm = previousClimateState['C_atm']
    C_ocean = previousClimateState['C_ocean']
    albedo = previousClimateState['albedo']
    time = previousClimateState['time']
    
    # Get the temperature anomaly implied by the carbon in the atmosphere and the albedo
    T_anomaly = CL.Diagnose_T_anomaly(C_atm, albedo, ClimateParams)
    actual_temperature = CL.Diagnose_actual_temperature(T_anomaly)
    OceanSurfacepH = CL.Diagnose_OceanSurfacepH(C_atm,ClimateParams)
    
    # Get new fluxes (including the effect of temperature anomaly on the ocean-to-atmosphere flux)
    F_la = k_la    
    F_al = k_al0 + k_al1*C_atm
    F_oa = k_oa*C_ocean*(1+DC*T_anomaly)
    F_ao = k_ao*C_atm

    # Get new concentrations of carbon that depend on the fluxes
    C_atm += (F_la + F_oa - F_ao - F_al + F_ha)*dt
    C_ocean += (F_ao - F_oa)*dt
    time += dt
    
    # Create a new climate state with these updates
    ClimateState = makeacopy(previousClimateState)
    ClimateState['C_atm'] = C_atm
    ClimateState['C_ocean'] = C_ocean
    ClimateState['F_al'] = F_al
    ClimateState['F_la'] = F_la
    ClimateState['F_ao'] = F_ao
    ClimateState['F_oa'] = F_oa
    ClimateState['F_ha'] = F_ha
    ClimateState['time'] = time
    ClimateState['T_anomaly'] = T_anomaly
    ClimateState['actual temperature'] = actual_temperature
    ClimateState['OceanSurfacepH'] = OceanSurfacepH
    ClimateState['albedo'] = albedo

    # Return the new climate state
    return ClimateState

# Make the starting state the preindustrial
ClimateState = CL.CreateClimateState(ClimateParams)
display(ClimateState)

# Initialize our list of climate states 
ClimateState_list = []

# The time interval
dt = epstime[1]-epstime[0]
print('time interval = ',dt)

# Loop over all the times after the initial one in the scheduled flow
for i in range(1,len(epstime)):

    # Propagate
    ClimateState = PropagateClimateState_Cambio2(ClimateState,ClimateParams,dt,eps[i])
    
    # Add to our list of climate states
    ClimateState_list.append(ClimateState)

# Extracting data from ClimateState_list
time_array = CL.CollectClimateTimeSeries(ClimateState_list,'time')
C_atm_array = CL.CollectClimateTimeSeries(ClimateState_list,'C_atm')
C_ocean_array = CL.CollectClimateTimeSeries(ClimateState_list,'C_ocean')
F_ao_array = CL.CollectClimateTimeSeries(ClimateState_list,'F_ao')
F_oa_array = CL.CollectClimateTimeSeries(ClimateState_list,'F_oa')
F_al_array = CL.CollectClimateTimeSeries(ClimateState_list,'F_al')
F_la_array = CL.CollectClimateTimeSeries(ClimateState_list,'F_la')
F_ha_array = CL.CollectClimateTimeSeries(ClimateState_list,'F_ha')
T_anomaly_array = CL.CollectClimateTimeSeries(ClimateState_list,'T_anomaly')
actual_temperature_array = CL.CollectClimateTimeSeries(ClimateState_list,'actual temperature')
OceanSurfacepH_array = CL.CollectClimateTimeSeries(ClimateState_list,'OceanSurfacepH')
albedo_array = CL.CollectClimateTimeSeries(ClimateState_list,'albedo')

# Reporting some key numbers
C_atm_Cambio_peak = np.max(C_atm_array)
print(plot_title+' has a maximum C_atm of', C_atm_Cambio_peak)
T_anomaly_Cambio_peak = np.max(T_anomaly_array)
print(plot_title+' has a maximum T_anomaly of', T_anomaly_Cambio_peak)

# Plotting the concentrations (C_atm and C_ocean) on one graph, in GtC
plt.figure()
plt.plot(time_array,C_atm_array,'red',label='C_atm')
plt.plot(time_array,C_ocean_array,label='C_ocean')
plt.grid(True)
plt.xlabel('time (years)')
plt.ylabel("GtC")
plt.title(plot_title)
plt.legend()

# Plot the atmospheric concentration (C_atm), in ppm (by dividing C_atm by 2.12)
plt.figure()
plt.plot(time_array,C_atm_array/2.12,'red',label='C_atm')
plt.grid(True)
plt.xlabel('time (years)')
plt.ylabel("ppm")
plt.legend()
plt.title(plot_title)

# Plot the net fluxes
plt.figure()
F_land_net = F_la_array-F_al_array
F_ocean_net = F_oa_array-F_ao_array
plt.plot(time_array,F_ha_array,label='F_ha',color='black')
plt.plot(time_array,F_land_net,label='F_la-F_al',color='brown')
plt.plot(time_array,F_ocean_net,label='F_oa-F_ao',color='blue')
plt.grid(True)
plt.xlabel('time (years)')
plt.ylabel("Flux differences (GtC/year)")
plt.legend()
plt.title(plot_title)

# Plot the temperature anomaly
plt.figure()
plt.plot(time_array,T_anomaly_array,label='T_anomaly',color='red')
plt.grid(True)
plt.xlabel('time (years)')
plt.ylabel("Temperature anomaly (K)")
plt.legend()
plt.title(plot_title)

# Plot the albedo
plt.figure()
plt.plot(time_array,albedo_array,label='albedo',color='blue')
plt.grid(True)
plt.xlabel('time (years)')
plt.ylabel("albedo")
plt.legend()
plt.title(plot_title)

# Plot the pH
plt.figure()
plt.plot(time_array,OceanSurfacepH_array,label='pH')
plt.grid(True)
plt.xlabel('time (years)')
plt.ylabel("Ocean Surface pH")
plt.legend()
plt.title(plot_title)


### Cambio 3.0
Below, the goal is to enhance Cambio2.0 with ice-albedo feedback. It'll be mostly a cut-and-paste job, but you'll need to make an adjustment in the function that propagates the climate state forward. It may help to keep things distinct, if you rename the propagating function to something like PropagateClimateState_Cambio3 (but make sure you also change that name where you invoke the function inside your Euler loop).

Make all the plots we did for Cambio2.0.

In [ ]:
# your code here 


### Pause for analysis
In the cell below, comment on some differences between Cambio 2.0 and 3.0, including ...
1. The peak temperature
1. The year in which oceans begin to return carbon to the atmosphere
1. The lowest pH reached
1. The lowest albedo reached

YOUR ANSWER HERE

### Refresh/save/validate
Double-check everything is OK, and press the "Validate" button (as usual).

### Close/submit/logout
Close, submit, and log out.